## Basic strategy overview:

* For each riddle, given its length n, letters will be guessed according to two possible schemes:

  * If no letter has been correctly guessed yet, letters will be guessed based on the frequency of letters in all n-long words in the dictionary.
  
  * If there have already been some correct guesses, an RNN model will be used to predict the next letter to be guessed.

## Table of contents.

### 1. Loading the existing dataset as a list.

In [3]:
with open('word_list.txt', 'r') as file:
    train_data = file.read().splitlines()

### 2. Creating a dictionary of most common letters in the words of each word length.

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
import random
from collections import defaultdict, Counter
from itertools import chain, combinations
from datetime import datetime

def find_most_common_letters(words):
    letters_by_length = defaultdict(Counter)

    for word in words:
        letters_by_length[len(word)].update(word.lower()) 
        
    most_common_letters = {length: [letter for letter, count in letters.most_common(6)]
                           for length, letters in letters_by_length.items()}
    return most_common_letters

most_common_letters = find_most_common_letters(train_data)

### 3. Create dataset for training

In order to train the RNN model that follows, synthetic riddles and their respective correct answers will be needed. These will be created according to the steps outlined below:

### 3.1. Creating synthetic riddles

For each word w in the training vocabulary, composed of a set k of unique letters, we can generate potential riddles. This is done by masking a subset of letters in k, where this subset is an element of the power set of k. Consequently, each element k_i from the power set of k represents a possible riddle, when all the letters in k_i are concealed in the riddle. For each riddle generated, a corresponding guess containing all the omitted letters is created, representing all the possible correct guesses. For example, in one iteration, a riddle "Co__ect_c_t" and the correct guess "niu" would be created.

The function `hangman_guesses(word,n)` below creates riddles from word word, limited to n riddles to avoid an excessive bias towardss words composed of a large number of different letters.  The cardinality of the power set being 2^{len(k)} means that without this limitation the RNN's exposure to shorter words would be restricted. In the example below, n=64.

Finally, a subset of 15% of all riddles and guesses were used for the following steps, to ensure feasible training times. This network was trained in an Apple M1 Pro 2021, 8-core, with 16GB RAM.

In [5]:
recreate_dataset = False
recreate_shuffled_dataset = False
recreate_shuffled_dataset_from_full = False

def substitute_letters(word, letters_to_omit):
    word_list = list(word)
    
    word_list = ["_" if char in letters_to_omit else char for char in word_list ]
    
    return ''.join(word_list)

#%%
def powerset(iterable):
    s = list(iterable)
    powerset = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
    powerset = [set(k) for k in powerset if (len(k) > 0 and len(k) < len(s))]
    random.shuffle(powerset)
    return powerset

#%%
def hangman_guesses(word, n):
    wordset = list(set(word))
    guesses = []
    changed_words = []
    subsets = powerset(wordset)
    for subset in subsets:
        changed_word = substitute_letters(word, subset)
        guesses.append(''.join(subset))
        changed_words.append(changed_word)
    return guesses[0:n], changed_words[0:n]

if recreate_dataset:
    all_guesses = []
    all_changed_words = []
    with open('all_guesses_limit64.txt', 'a') as guesses_file, open('all_changed_words_limit64.txt', 'a') as changed_words_file:
        for i, word in enumerate(train_data):
            if i % 1000 == 0:
                print(f"Processing word {i}")
            guesses, changed_words = hangman_guesses(word, 64)
            
            # Write the guesses and changed words to their respective files
            for guess in guesses:
                guesses_file.write(guess + '\n')
            
            for changed_word in changed_words:
                changed_words_file.write(changed_word + '\n')

if recreate_shuffled_dataset_from_full:
    with open('all_changed_words.txt', 'r') as file:
        riddles_full = file.read().splitlines() 
        
    with open('all_guesses.txt', 'r') as file:
        guesses_full = file.read().splitlines() 
    lg = len(guesses_full)
    idxs = random.sample(range(0, lg), int(lg*0.15))
    print("Number of samples: ", len(idxs))
    riddles = [riddles_full[i] for i in idxs]
    print("Number of samples: ", len(riddles))
    guesses = [guesses_full[i] for i in idxs]
    print("Number of samples: ", len(guesses))
    print("Writing riddles and guesses to file")
    with open('riddles15.txt', 'w') as file:
        for riddle in riddles:
            file.write(riddle + '\n')
    print("Writing guesses to file")
    with open('guesses15.txt', 'w') as file:
        for guess in guesses:
            file.write(guess + '\n')

if recreate_shuffled_dataset:
    with open('all_changed_words_limit64.txt', 'r') as file:
        riddles_full = file.read().splitlines() 
        
    with open('all_guesses_limit64.txt', 'r') as file:
        guesses_full = file.read().splitlines() 
    lg = len(guesses_full)
    idxs = random.sample(range(0, lg), lg)
    print("Number of samples: ", len(idxs))
    riddles = [riddles_full[i] for i in idxs]
    print("Number of samples: ", len(riddles))
    guesses = [guesses_full[i] for i in idxs]
    print("Number of samples: ", len(guesses))
    print("Writing riddles and guesses to file")
    with open('riddles_limit64.txt', 'w') as file:
        for riddle in riddles:
            file.write(riddle + '\n')
    print("Writing guesses to file")
    with open('guesses_limit64.txt', 'w') as file:
        for guess in guesses:
            file.write(guess + '\n')
else:    
    with open('riddles_limit64.txt', 'r') as file:
        riddles = file.read().splitlines()

    with open('guesses_limit64.txt', 'r') as file:
        guesses = file.read().splitlines()       

### 3.2. Preprocessing the data for the RNN

#### 3.2.1. Preprocessing riddles

Each riddle underwent TextVectorization, converting each of the possible characters in the riddle to an integer. Each letter is an object of interest, so riddles were split by character. Riddle length was limited to 15 characters to accommodate the majority of the words in the training set.

#### 3.2.2. Preprocessing guesses

This stage maps each guess, e.g. "eiu" in "Conn_ct_c_t" to a 1 x 26 vector of probabilities (here, [0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]), corresponding to the 26 letters of the alphabet. This encoding employs `tf.keras.layers.StringLookup` for multi-hot encoding.

The following steps involved:

- Removing the first element from each encoded guess, which corresponds to the "Unknown" character. This character is present in all encoded guesses and does not contribute information to the RNN.

- Converting both guesses and riddles into tensor slices.

Each element of the final dataset is a 2-tuple with an encoded riddle and an encoded guess. Samples are processed in 64-unit batches. The prefetching of one batch is added to improve performance.

Finally, the dataset is split into a train-set and a test-set, containing respectively 80% and 20% of the data. 

This setup was demonstrated using the following code snippet:

In [6]:
text_vec_layer = tf.keras.layers.TextVectorization(output_sequence_length = 15, # Max length of words
                                                   output_mode = "int",
                                                   split = "character")
text_vec_layer.adapt(riddles[1:10000])
print("Vocabulary size: ", text_vec_layer.vocabulary_size())

# Convert words to lists of characters
char_lists = [list(word) for word in guesses]

# Pad the sequences to have the same length
padded_char_lists = tf.keras.preprocessing.sequence.pad_sequences(char_lists,
                                                                  dtype=object, 
                                                                  padding='post',
                                                                  value='')

lookup_layer = tf.keras.layers.StringLookup(vocabulary=list("abcdefghijklmnopqrstuvwxyz"),
                                            output_mode='multi_hot',
                                            pad_to_max_tokens=True,
                                            max_tokens=27, 
                                            sparse=True)
print("Encoding riddles")
encoded_riddles = text_vec_layer(riddles)
print("Encoding guesses")

encoded_guesses = lookup_layer(padded_char_lists)
print("Encoding complete")
encoded_guesses = tf.sparse.to_dense(encoded_guesses)
encoded_guesses = encoded_guesses[:,1:]
tensor_riddles = tf.data.Dataset.from_tensor_slices(encoded_riddles)
tensor_guesses = tf.data.Dataset.from_tensor_slices(encoded_guesses)
dsmap = tf.data.Dataset.zip(tensor_riddles, tensor_guesses)
dsmap = dsmap.batch(64)
dsmap = dsmap.prefetch(1)

#%%
def is_test(x, _):
    return x % 5 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x, y: y

# Split the dataset for training.
test_dataset = dsmap.enumerate().filter(is_test).map(recover)

# Split the dataset for testing/validation.
train_dataset = dsmap.enumerate().filter(is_train).map(recover)

Vocabulary size:  28
Encoding riddles and guesses
Encoding riddles
Encoding guesses
Encoding complete


## 4. RNN model

An RNN model is constructed to estimate the best guess for each riddle.

- The number of tokens is set to 28, which is equal to the 26 letters in the English language and two additional tokens (underscore sign and "unknown")

- Initially, the RNN model is composed of the 256-cell long bidirectional recurrent layers. These bidirectional layers capture dependencies in both directions of the input sequence, which is important to understand the context around missing letters in hangman. 

- The recurrent dropout of 0.1 in both GRU layers and the subsequent dropout of 0.1 in the following layer are used to prevent overfitting, thus allowing for better performance out of sample (which is especially important since hangman will be played with unknown words)

- The dense layer of 128 units and ELU activation provides nonlinearity to the model and helps to learn complex patterns from the data. ELU handles the vanishing gradient problem faster than ReLU and shows faster convergence in the data as well.

- The final Dense layer has 26 units, corresponding to the 26 letters of the alphabet, with softmax activation to output a probability distribution over all possible letters.  

- The model is initially set to run through a maximum of 100 epochs. EarlyStopping is employed to prevent overfitting by halting training if the model's loss doesn't improve after three epochs, starting from epoch 2, restoring the weights from its best-performing iteration.

- The model is optimized using the Nadam optimizer and binary crossentropy loss; which will be minimized as the vector of predicted probabilities approaches the vector of true probabilities mentioned in Section 3.2.2. The precision metric is configured to evaluate the model's performance, specifically measuring the precision of the prediction with highest probability, which is critical in a game like hangman where the first guess significantly influences game progress.

- Several hyperparameters were calibrated (number of cells in each layer; number of layers; dropout rates). Training time were also considered.

- The model was fit with the trainset, whereas performance is evaluated in unseen riddles (`test_dataset`), since generalizing well to unseen data is fundamental here. The number of `steps_per_epoch` was limited to 25,000 for increased computation speed.

In [ ]:
n_tokens = 28
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(n_tokens, output_dim=28))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, recurrent_dropout = 0.1, return_sequences=True))) 
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, recurrent_dropout = 0.1, return_sequences=False)))
model.add(tf.keras.layers.Dropout(0.1))  
model.add(tf.keras.layers.Dense(128, activation='elu')) 
model.add(tf.keras.layers.Dense(26, activation='softmax'))
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, start_from_epoch=2, restore_best_weights=True)
model.compile(optimizer=tf.keras.optimizers.Nadam(), loss="binary_crossentropy",  metrics=[tf.keras.metrics.Precision(top_k=1, name="First_precision")])    
model.fit(train_dataset, validation_data = test_dataset, epochs=100, steps_per_epoch=25000, callbacks=[callback])

The following snippet tests whether the first guess found by model is correct in a subset of riddles.

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz"
def first_guess_right(maxplays = 1000, model = model, guesses = guesses, riddles = riddles, text_vec_layer = text_vec_layer):
    total_correct = 0
    for i in range(0, maxplays):
        word = riddles[-i]
        guess = guesses[-i]
        right_choices = list(guess)
        x = model.predict(text_vec_layer([word]))
        letter = alphabet[tf.argmax(x[0])]
        if letter in right_choices:
            total_correct += 1
    return total_correct/maxplays
first_guess_right()